# Correction décorateurs

Durée : 1 heure
Langage : Python 3.x

Sujet : Les décorateurs en Python sont un moyen pratique de modifier le comportement d’une fonction sans en altérer le code source directement. Ils sont utiles pour la journalisation, la validation, le contrôle d’accès, etc.


## 1. Décorateur de Journalisation Simple

- Écrire un décorateur @journaliser qui, appliqué à une fonction, affiche :
  - Le nom de la fonction appelée
  - Les arguments positionnels et nommés passés à cette fonction
  - La fonction décorée doit ensuite s’exécuter normalement et retourner son résultat.

Exemple d’utilisation :

```py
@journaliser
def addition(a, b):
    return a + b
```

Appeler `addition(3, 5)` devrait afficher quelque chose comme :

`Appel de addition(3, 5)`

Et retourner 8.

In [13]:
def journaliser(fonction):
    def wrapper(*args, **kwargs):
        # Construction d'une représentation claire des arguments
        args_repr = ", ".join(repr(a) for a in args)                # ex : "3, 5"
        kwargs_repr = ", ".join(f"{k}={v!r}" for k, v in kwargs.items())  # ex : "x=2, y=4"
        
        # Assemblage pour l'affichage
        if args_repr and kwargs_repr:
            signature = args_repr + ", " + kwargs_repr
        elif args_repr:
            signature = args_repr
        else:
            signature = kwargs_repr
        
        # Journalisation
        print(f"Appel de {fonction.__name__}({signature})")
        
        # Appel de la fonction d'origine
        resultat = fonction(*args, **kwargs)
        
        return resultat
    
    return wrapper

# --- Test / Démo ---
@journaliser
def addition(a, b):
    return a + b

In [14]:
print("Résultat :", addition(3, 5))  
# Doit afficher :
# Appel de addition(3, 5)
# Résultat : 8

Appel de addition(3, 5)
Résultat : 8


## 2. Décorateur de Mémorisation (Caching)
- Écrire un décorateur @memoriser qui stocke les résultats de la fonction décorée en fonction de ses arguments.
- Si la fonction est appelée à nouveau avec les mêmes arguments, retourner directement le résultat mémorisé au lieu de recalculer.
- Tester ce décorateur sur une fonction de calcul un peu coûteuse (par exemple, une fonction qui calcule le n-ième nombre de Fibonacci de manière récursive).

In [27]:
def memoriser(fonction):
    """
    Décorateur qui stocke les résultats de la fonction
    en fonction de ses arguments pour éviter de recalculer.
    """
    cache = {}  # Dictionnaire pour la mémorisation
    
    def wrapper(*args, **kwargs):
        # On crée une clé immuable (tuple) basée sur args + un frozenset de kwargs
        key = (args, frozenset(kwargs.items()))
        
        # Explication: En Python, un set est une collection d’éléments non ordonnée et modifiable (on peut ajouter, retirer des éléments). 
        # Comme il est modifiable, on ne peut pas l’utiliser comme clé dans un dictionnaire (car les clés doivent être immuables, donc hachables). 
        # Un frozenset, c’est la version immutable (figée) d’un set. On ne peut plus le modifier après sa création. 
        # Ainsi, Python autorise un frozenset à être utilisé comme clé dans un dictionnaire (ou placé dans un set) parce qu’il est hachable.
        
        # Si la clé est dans le cache, on renvoie directement la valeur
        if key in cache:
            print("Résultat en cache pour", key)  # Pour tester
            return cache[key]
        
        # Sinon, on calcule la fonction et on stocke le résultat
        resultat = fonction(*args, **kwargs)
        cache[key] = resultat
        
        return resultat
    
    return wrapper


# --- Démo sur une fonction coûteuse : Fibonacci récursif ---
@memoriser
def fibonacci(n):
    """
    Calcul du n-ième Fibonacci de manière récursive (très coûteuse sans mémorisation).
    """
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [28]:
# On peut tester sur des valeurs un peu grandes
print("fibonacci(10) =", fibonacci(10))  # 55
print("fibonacci(20) =", fibonacci(20))  # 6765

Résultat en cache pour ((1,), frozenset())
Résultat en cache pour ((2,), frozenset())
Résultat en cache pour ((3,), frozenset())
Résultat en cache pour ((4,), frozenset())
Résultat en cache pour ((5,), frozenset())
Résultat en cache pour ((6,), frozenset())
Résultat en cache pour ((7,), frozenset())
Résultat en cache pour ((8,), frozenset())
fibonacci(10) = 55
Résultat en cache pour ((10,), frozenset())
Résultat en cache pour ((9,), frozenset())
Résultat en cache pour ((10,), frozenset())
Résultat en cache pour ((11,), frozenset())
Résultat en cache pour ((12,), frozenset())
Résultat en cache pour ((13,), frozenset())
Résultat en cache pour ((14,), frozenset())
Résultat en cache pour ((15,), frozenset())
Résultat en cache pour ((16,), frozenset())
Résultat en cache pour ((17,), frozenset())
Résultat en cache pour ((18,), frozenset())
fibonacci(20) = 6765


## 3. Combinaison de décorateurs
Appliquer les deux décorateurs @journaliser et @memoriser à la même fonction pour observer l’ordre d’exécution (décorer la fonction avec @journaliser puis @memoriser, ou l’inverse, et commenter le comportement).


In [24]:
@journaliser
@memoriser
def addition_memorisee(a, b):
    print("  [Corps de la fonction addition_memorisee exécuté]")
    return a + b

print(addition_memorisee(3, 5))
print(addition_memorisee(3, 5))

Appel de wrapper(3, 5)
  [Corps de la fonction addition_memorisee exécuté]
8
Appel de wrapper(3, 5)
Résultat en cache pour (3, 5)
8


Que se passe-t-il ?

1.	Lors du premier appel addition_memorisee(3, 5) :
    -	La fonction la plus externe (due à @journaliser) s’exécute et affiche :
    Appel de wrapper((3, 5)) (ou quelque chose dans ce style, selon l’implémentation).
    -	À l’intérieur, on appelle la fonction renvoyée par memoriser (c’est-à-dire le wrapper de memoriser).
    -	Comme c’est la première fois qu’on appelle (3, 5), il n’y a pas de résultat dans le cache, donc on exécute le vrai addition_memorisee.
    -	On affiche : [Corps de la fonction addition_memorisee exécuté], et on retourne 8.
    -	Tout remonte et 8 est renvoyé et affiché.
2.	Lors du deuxième appel addition_memorisee(3, 5) :
    -	De nouveau, @journaliser affiche : Appel de wrapper((3, 5)).
    -	On arrive dans le memoriser, qui cette fois reconnaît (3, 5) dans son cache.
    -	Il ne ré-exécute pas le corps de addition_memorisee.
    -	Il renvoie directement 8.

Résultat : on journalise toujours l’appel (à cause de @journaliser), mais on ne recalcule pas la somme (grâce à @memoriser).

In [ ]:
# On inverse

@memoriser
@journaliser
def addition_memorisee_inverse(a, b):
    print("  [Corps de la fonction addition_memorisee_inverse exécuté]")
    return a + b

-	Cette fois, l’ordre est différent :
    -	On applique d’abord @journaliser.
    -	Puis, le résultat de journaliser(...) est décoré par @memoriser.
-	Ce qui veut dire qu’on va faire le « journal » avant de regarder si le résultat est en cache.
-	On observera que la journaling se produit peut-être à chaque appel, même si le résultat est en cache, selon l’implémentation choisie.

En conclusion, l’ordre des décorateurs a un impact direct sur la façon dont les décorateurs communiquent et à quel moment s’appliquent leurs effets.